# 0. 环境准备

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 设置pandas可以显示的行数和列数
pd.options.display.max_rows = 400
pd.options.display.max_columns = None

# 忽略warnings
import warnings
warnings.filterwarnings("ignore")

#推荐安装插件： nbextensions

# 1.读入数据

In [2]:
# store:门店编号
# dept: 商品部门编号
# week: 每周周一的日期 
# sales: 销售金额
df_sales = pd.read_csv('data/store_sales.csv', parse_dates=['week'])
df_sales.head(2)

store  dept       week     sales
0      1     1 2010-02-01  24924.50
1      1     1 2010-02-08  46039.49

In [3]:
# store:门店编号
# dept: 商品部门编号
# week: 每周周一的日期 
# promotion_sales: 促销活动带来的销售金额
df_promotion = pd.read_csv('data/promotion_data.csv', parse_dates=['week'])
df_promotion.head(2)

store  dept       week  promotion_sales
0      1     1 2010-02-08     22538.074165
1      1     1 2010-02-15     18381.721909

In [4]:
# 合并销售和促销数据
df_all = pd.merge( df_sales, df_promotion, how='left' )
df_all.fillna(0, inplace=True)

# 2. 用Prophet算法预测所有门店、所有部门预测2012-07-30及以后的销量，每次往后预测1周

In [5]:
import prophet

Importing plotly failed. Interactive plots will not work.


In [ ]:
# 对门店、部门和预测日期进行for循环
fcst_date_list = pd.date_range( '2012-07-30', df_all['week'].max() - pd.Timedelta(7, unit='D'), freq='7D' )
store_list = df_all['store'].unique()
dept_list = df_all['dept'].unique()

prophet_result = []
for fcst_date in fcst_date_list:
    for store in store_list:
        for dept in dept_list:
            df_train = df_all[ (df_all['week']<=fcst_date) & 
                              (df_all['store']==store) & 
                              (df_all['dept']==dept)]
            df_train.rename(columns={'week':'ds','sales':'y'},inplace=True)

            df_test = df_all[ (df_all['week']==fcst_date+pd.Timedelta(7, unit='D')) &
                             (df_all['store']==store) & 
                             (df_all['dept']==dept)]
            df_test.rename(columns={'week':'ds','sales':'y'},inplace=True)

            #只有在训练集超过100条、测试集超过1条的情况下才做预测
            if (df_train.shape[0] > 100) & (df_test.shape[0] > 0):
                #训练模型
                m = prophet.Prophet(yearly_seasonality=True)
                m.add_regressor( 'promotion_sales' )
                m.fit( df_train )

                #预测结果
                df_predict = m.predict( df_test )
                df_test['yhat'] = df_predict['yhat'].values
                df_test['fcst_date'] = fcst_date
                prophet_result.append( df_test )
prophet_result = pd.concat( prophet_result )

In [13]:
# 计算mape
prophet_result['percentage_error'] = abs(prophet_result['yhat']-prophet_result['y'])/prophet_result['y']
#通常我们用y值作为权重，计算加权平均误差
print("mape", np.sum( prophet_result['percentage_error']*prophet_result['y'] )/np.sum( prophet_result['y'] ) ) 

mape 0.0476318528867222


# 3. 用LightGBM算法预测所有门店、所有部门预测2012-07-30及以后的销量，每次往后预测1周

In [19]:
from lightgbm.sklearn import LGBMRegressor

In [15]:
df_sample = df_all.sort_values(['dept','week'])

In [16]:
# 特征构建
feature_cols = []

## 第一组特征： 历史数据最后一周的销量和促销活动的金额
df_sample['sales_lw'] = df_sample.groupby(['store','dept'])['sales'].shift(1)
df_sample['promotion_lw'] = df_sample.groupby(['store','dept'])['promotion_sales'].shift(1)
feature_cols = feature_cols + ['sales_lw', 'promotion_lw']

## 第二组特征： 上一个周期（即去年同一周）的销量和促销活动金额
df_sample['sales_ly'] = df_sample.groupby(['store','dept'])['sales'].shift(52)
df_sample['promotion_ly'] = df_sample.groupby(['store','dept'])['promotion_sales'].shift(52)
feature_cols = feature_cols + ['sales_ly', 'promotion_ly']

## 第三组特征：待预测周的促销活动金额
feature_cols = feature_cols + ['promotion_sales']

## 只保留所有特征都不为空的数据
for col in feature_cols:
    df_sample = df_sample[ ~df_sample[col].isna() ]

In [17]:
# 构建训练集和验证集
x_train = df_sample[ df_sample['week']<='2012-07-30' ][ feature_cols ].values
y_train = df_sample[ df_sample['week']<='2012-07-30' ][ 'sales' ].values

x_test = df_sample[ df_sample['week']>='2012-08-06' ][ feature_cols ].values
y_test = df_sample[ df_sample['week']>='2012-08-06'  ][ 'sales' ].values

In [20]:
# 使用lightGBM建模
model = LGBMRegressor()
model.fit(x_train, y_train)

LGBMRegressor()

In [28]:
# 预测
y_pred = model.predict( x_test )

lgb_result = df_sample[ df_sample['week']>='2012-08-06'  ]
lgb_result['pred_sales'] = y_pred

In [30]:
# 计算mape
lgb_result['percentage_error'] = abs(lgb_result['pred_sales']-lgb_result['sales'])/lgb_result['sales']
#通常我们用y值作为权重，计算加权平均误差
print("mape", np.sum( lgb_result['percentage_error']*lgb_result['sales'] )/np.sum( lgb_result['sales'] ) ) 

mape 0.06182162662713885
